# URL Classifier by SVM

**Step 0: Importing librarys and packages**

In [439]:
!pip install pandas
!pip install regex
!pip install re
!pip install matplotlib
!pip install nltk
!pip install sklearn
!pip install wordsegment


[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [440]:
import pandas as pd
import regex as reg
import re
import matplotlib.pyplot as plt
import unicodedata
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection, svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
nltk.download('stopwords')
from wordsegment import load, segment

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Step 1 : Load the data**

In [441]:
url='https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv'
df_raw = pd.read_csv(url)

In [442]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [443]:
df_raw.sample(10)

,url,is_spam
423,https://flowingdata.com/2020/06/22/age-generat...,False
1443,https://www.youtube.com/watch?v=7fHi3djOHl0,False
1787,https://bitchesgottaeat.substack.com/,False
1967,https://www.clarionledger.com/story/news/2020/...,False
2241,https://www.morningbrew.com/daily/latest,False
1913,https://benjaminreinhardt.com/wddw,False
2361,https://deceuvel.nl/en/about/sustainable-techn...,False
1600,https://www.morningbrew.com/daily/stories/2020...,False
1712,https://www.ben-evans.com/newsletter,True
1562,https://theconversation.com/northern-irelands-...,False


The database has only two variables of which ones is an object and it seems to be a string. The other variable is a boolean and it seems to be the target that classify the URL between spam and recular URL.

Next i am going to change the data type of the target variable to facilitate the encoding.

In [444]:
df_interim=df_raw.copy() # copy of df_raw to make the processing

In [445]:
df_interim['is_spam']=df_interim['is_spam'].astype('category')

**Encoding**

In [446]:
df_interim['is_spam']=df_interim['is_spam'].cat.codes

In [447]:
df_interim.sample(10)

,url,is_spam
2140,https://thehustle.co/06302020-Zwift-training/,0
1049,https://theneed2know.us2.list-manage.com/about,1
1472,https://austinkleon.com/newsletter/,1
1092,https://people.com/country/dixie-chicks-change...,0
1772,https://www.nbcnews.com/news/us-news/saharan-d...,0
1617,https://www.morningbrew.com/daily/r/,1
432,https://apnews.com/4dcff8f062bae9e1fe3885c346b...,0
1730,https://thekanyestory.com/,0
1039,https://www.esquire.com/uk/culture/a32848474/v...,0
2441,https://www.densediscovery.com/issues/94#comments,0


**Setep 2: Preprocess the data with NLP techniques**

In [448]:
df_interim.nunique()

url        2369
is_spam       2
dtype: int64

There is some dupliquetade url, so i delete it.

In [449]:
df_interim.drop_duplicates(inplace=True)
df_interim.reset_index(inplace=True)

In [450]:
df_interim.shape

(2369, 3)

In [451]:
df_interim.sample(10)

,index,url,is_spam
1753,2099,https://thehustle.co/06292020-Amazon-vehicles/,0
469,543,https://apnews.com/4565b519d6b7d0a673105767148...,0
541,642,https://www.cbsnews.com/news/fauci-coronavirus...,0
756,901,https://www.theguardian.com/us-news/2020/jun/2...,0
944,1129,https://en.wikipedia.org/wiki/Zuihitsu,0
1488,1764,https://cheddar.com/media/stocks-sink-coronavi...,0
1622,1945,https://www.nytimes.com/2020/06/24/us/confeder...,0
1799,2158,https://subscribe.washingtonpost.com/newslette...,1
561,667,https://www.morningbrew.com/daily/stories/2020...,0
1016,1207,https://www.imperica.com/en/,1


Some things i can detect at plain sight:

- strange symbols that has been remove

- the '-' seems to separate words on the url

- https and www have to be remove

In [452]:
df_interim['url'] = df_interim['url'].str.replace(r'-', ' ', regex=True)

In [453]:
df_interim['url']=df_interim['url'].str.strip() # delet the firt and last space
df_interim['url']=df_interim['url'].str.lower() # change all the text to lower
# clean up strange symbols and standardize 
df_interim["url"] = df_interim["url"].apply(lambda x: re.sub(r'www'," ",x))
df_interim['url'] = df_interim['url'].str.replace('''[?&#,;ü_=%./']''','',regex=True)
df_interim['url'] = df_interim['url'].str.replace(r'http(s):', '', regex=True)
df_interim['url'] = df_interim['url'].str.replace(r'http:', '', regex=True)
df_interim['url'] = df_interim['url'].str.replace(r'com', '', regex=True)
# remove all numbers
df_interim['url'] = df_interim['url'].str.replace(r'[\d-]', '', regex=True) 
# remove all single letters
df_interim["url"] = df_interim["url"].apply(lambda x: re.sub(r'\b[a-zA-Z]\b'," ",x))
# remove characters repited more than two times
df_interim['url']=df_interim['url'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)

In [454]:
# remove stopwords
stop=stopwords.words('english')

def remove_stopwords(message):
    if message is not None:
        words = message.strip().split()
        words_filtered = []
        for word in words:
            if word not in stop:
                words_filtered.append(word)
                result = " ".join(words_filtered)
    else:
        result = None

    return result

In [455]:
df_interim['url']=df_interim['url'].apply(remove_stopwords)

To control I display first 60 words separately to see if they were cleaned up.

In [456]:
df_interim['url'].str.split(expand=True).stack().value_counts()[:60]

coronavirus                                    107
new                                             62
reutersarticleus                                59
trump                                           54
covid                                           51
us                                              50
sunday                                          45
numlocksubstackpnumlock                         42
court                                           38
cases                                           31
police                                          29
black                                           27
facebook                                        27
pandemic                                        25
world                                           23
says                                            23
usa                                             21
live                                            21
year                                            20
york                           

In [457]:
df_interim.sample(10)

,index,url,is_spam
956,1142,youtubewatchvrivqdzhq,0
2040,2467,docsgoogledocumentdykvrkybxbuvkkn yedxxosnaqwd...,1
1050,1250,insidehookarticlepoliticsedgy edy lazy joe rog...,0
716,851,latimescaliforniastory george floyd carotid ne...,0
575,686,lunyacocollectionshey skimmrs heres use code s...,0
2131,2695,ewtvcurb enthusiasm renewed season,0
1341,1593,cnbcamazon buys self driving technology pany z...,0
1242,1472,austinkleonnewsletter,1
2061,2602,morningbrewdaily,0
1905,2295,theskimmpicks hottest books season ofbwmpvfwyw...,1


It seems that some url don't has space between different words, so it would be dificult for the algorithm to learn from it. Therefore, it is necessary to apply word segmentation.

I am going to apply an algorithm that I have found in: https://grantjenks.com/docs/wordsegment/#:~:text=WordSegment%20is%20an%20Apache2%20licensed,Segaran%20and%20Hammerbacher%2C%202009).

In [458]:
load() # reads and parses the unigrams and bigrams data from disk

In [459]:
# Function that makes words segmentarion to each row
def segment_words(message):
    if message is not None:
        words = segment(message)
        words_filtered = []
        for word in words:
            if word not in stop: # if any word is in stopwords, it removes
                words_filtered.append(word)
                result = " ".join(words_filtered)
    else:
        result = None

    return result

In [460]:
df_interim['url']=df_interim['url'].apply(segment_words)

In [461]:
df_interim.sample(10)

,index,url,is_spam
443,514,verge microsoft gaming cloud xbox future,0
2238,2828,numlock sub stack p numlock sunday olga khaz w...,1
1156,1383,wordpress creative mornings,1
1058,1259,story california sunday eleanor davis hugs loo...,0
2349,2977,venture beat ibm uses enhance old wimbledon te...,0
397,445,cnn tech wire card missing money index html,0
879,1054,week articles seattle police free autonomous z...,0
436,507,us four sigma tic summer sale,0
720,856,cnet news chicken run sequel ing netflix,0
411,466,chocolate basil squarespace blog pretzel short...,0


Now that the url is cleaner, i select the target and feature variable and split the original sample between training and test sample.

In [462]:
df_final=df_interim.copy()

In [463]:
y=df_final['is_spam']
X=df_final['url']

In [464]:
y.value_counts(normalize=True)

0    0.897003
1    0.102997
Name: is_spam, dtype: float64

The sample is unbalanced, this must be taken into account when applying the model.

In [465]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2007,stratify=y)

**Tokenization**

Transform the body of text into a sparse matrix of numbers that the computer can pass to machine learning algorithms.

In [466]:
vec=CountVectorizer(stop_words='english')
X_train=vec.fit_transform(X_train).toarray()
X_test=vec.transform(X_test).toarray()

In [467]:
X_train.shape

(1776, 4720)

We can see that there are 4720 columns that refers to the different unique words inside each url. Now we could see the first ten unique words.

In [468]:
vec.get_feature_names_out()[:10]

array(['aaae', 'aadac', 'aafc', 'aah', 'ab', 'abacus', 'abba', 'abbott',
       'abbreviated', 'abc'], dtype=object)

**SVM classifier**

In [469]:
clf_svm = svm.SVC(C=1.0, kernel='linear', degree=3,random_state=3107, gamma='auto',class_weight='balanced') 

In [470]:
clf_svm.fit(X_train, y_train) 
y_pred = clf_svm.predict(X_test)
print('Accuracy scores')
print(classification_report(y_test, y_pred))

Accuracy scores
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       532
           1       0.49      0.70      0.58        61

    accuracy                           0.89       593
   macro avg       0.73      0.81      0.76       593
weighted avg       0.92      0.89      0.90       593



The accurancy of the model is 89% which is very good! Indeed the recall almost reach the 70% for the minority class.

**Plus: Naive Bayes classifier**

In [471]:
nb=MultinomialNB()
nb.fit(X_train,y_train)
nb_y_pred=nb.predict(X_test)
print(classification_report(y_test,nb_y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       532
           1       0.75      0.62      0.68        61

    accuracy                           0.94       593
   macro avg       0.85      0.80      0.82       593
weighted avg       0.94      0.94      0.94       593



Among both classifiers, SVM algorithm has better recall ratio but worst accuracy than Naive Bayes

In [472]:
import pickle
filename = '/workspace/NLP/models/finalized_model.sav'
pickle.dump(clf_svm, open(filename, 'wb'))